Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

### Load Competition Data

In [1]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lambda DS/04.Unit_4/01.NLP/03.Document_Classification/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lambda DS/04.Unit_4/01.NLP/03.Document_Classification/test.csv')

In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [4]:
train.shape

(4087, 3)

In [5]:
train['description'][10]

'\nThis has the classic sweetish, oily, floral, leathery nose of 100% rye grain whisky. A luxurious, creamy, medium-weight dram with caraway rye bread, restrained but hot peppery spices, vague citrus notes, and hints of cedar setting goal posts for punts of sweet and vaguely floral barley sugar, nail polish (in a good way), caramel, barrel wood, well-aged lumber, dark rye bread, ripe apples, milk chocolate, and baking spices. $90 CAD'

In [3]:
def clean_data(text):
  """
  Cleans data to remove unwanted characters and punctuation.
  """
  # remove newline and '\xa0' characters
  text = " ".join(text.split())
  # remove punctuation
  text = re.sub('[^a-zA-Z$ 0-9]', '', text)
  # remove any double whitespaces created by cleaning
  text = re.sub("[ ]{2,}", ' ', text)

  return text.lower()

In [4]:
# Clean train and test dataframes
train['description'] = train['description'].apply(clean_data)
test['description'] = test['description'].apply(clean_data)

In [16]:
X_train = train['description']
y_train = train['ratingCategory']
X_test = test

In [18]:
print(X_train.shape)
print(y_train.shape)

(4087,)
(4087,)


In [21]:
pipe = Pipeline([('vect', TfidfVectorizer(stop_words='english')),
                 ('clf', RandomForestClassifier())])

param_grid = {
    'vect__max_df': [.75, 1.0],
    'clf__max_depth': [5,10,15,20]
}

grid_search = GridSearchCV(pipe, param_grid, cv=3, n_jobs=-2, verbose=1)

# fit me the model and do cross validation
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done  24 out of  24 | elapsed:   15.7s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        no

In [22]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

{'clf__max_depth': 20, 'vect__max_df': 1.0}
Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_patte...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=20, max_features='auto',
                                   

### Define Pipeline Components

In [26]:
vect = TfidfVectorizer(stop_words='english')
clf = XGBClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [27]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20),
    'clf__eta':(0.3, 0.5, 0.8)
}

grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed: 31.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        no

In [28]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

{'clf__eta': 0.3, 'clf__max_depth': 15, 'vect__max_df': 0.75}
Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.75, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eta=0.3, gamma=0,
                               lea

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [29]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [30]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [31]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [32]:
subNumber = 0

In [35]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'/content/drive/MyDrive/Colab Notebooks/Lambda DS/04.Unit_4/01.NLP/03.Document_Classification/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

### Define Pipeline Components

In [14]:
svd = TruncatedSVD(n_components = 100) # Generate 100 topics from the tokens
vect = TfidfVectorizer(stop_words='english') # Vectorize the description
clf = XGBClassifier()

lsa = Pipeline([('vect', vect), ('svd', svd)]) # Create LSA pipeline

pipe = Pipeline([('lsa', lsa), ('clf', clf)]) # Create nested pipeline with XGBoost Classifier

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [41]:
parameters = {
    'lsa__svd__n_components': [10,100,250],
    'lsa__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed: 17.7min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsa',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [42]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

{'clf__max_depth': 10, 'lsa__svd__n_components': 250, 'lsa__vect__max_df': 0.75}
Pipeline(memory=None,
         steps=[('lsa',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=0.75,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 1), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                               

### Make a Submission File

In [43]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [44]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [45]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [46]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'/content/drive/MyDrive/Colab Notebooks/Lambda DS/04.Unit_4/01.NLP/03.Document_Classification/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [5]:
import spacy
# Load pre-trained word2vec
nlp = spacy.load('en_core_web_lg')

In [10]:
def get_word_vectors(docs):
  """Tokenizes and vectorizes the documents"""
  return [nlp(doc).vector for doc in docs]

In [56]:
X_train_w2v = get_word_vectors(train['description'])
X_test_w2v = get_word_vectors(test['description'])

In [61]:
svd = TruncatedSVD(n_components = 100) # Generate 100 topics from the tokens
vect = TfidfVectorizer(stop_words='english') # Vectorize the description
clf = RandomForestClassifier()

pipe = Pipeline([('svd', svd), ('clf', clf)]) # Create nested pipeline with XGBoost Classifier

In [62]:
pipe.fit(X_train_w2v, y_train)

Pipeline(memory=None,
         steps=[('svd',
                 TruncatedSVD(algorithm='randomized', n_components=100,
                              n_iter=5, random_state=None, tol=0.0)),
                ('clf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, war

In [63]:
pipe.score(X_train_w2v, y_train)

0.9995106435037925

In [57]:
type(X_train_w2v)

list

In [58]:
type(y_train)

pandas.core.series.Series

In [53]:
X_train_w2v.shape

(4087,)

In [60]:
X_train_w2v

Output hidden; open in https://colab.research.google.com to view.

In [59]:
pd.DataFrame(data=X_train_w2v).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.036574,0.129909,-0.148589,-0.076406,0.125217,0.020135,-0.091932,-0.022086,0.006418,1.657298,-0.086580,0.031978,-0.098074,-0.132483,-0.019267,-0.155596,-0.024553,0.925815,-0.047376,-0.067164,-0.000415,-0.062061,-0.016529,0.012961,-0.121247,-0.075861,-0.024029,0.046979,0.020519,-0.140840,-0.036263,0.092131,0.104716,-0.097506,-0.012884,0.022277,0.019930,-0.062871,-0.102042,0.021052,...,0.109175,0.016771,-0.035779,0.087517,0.006749,0.021869,-0.059565,-0.031722,-0.027108,-0.005258,-0.033104,-0.052871,-0.002955,-0.163980,-0.146396,-0.008046,0.040656,0.078292,-0.031668,0.239786,0.150570,0.125027,0.025614,-0.046308,-0.031820,0.063501,0.205800,-0.011216,0.104816,-0.033094,-0.183097,0.012958,0.001837,-0.017301,-0.080962,0.005669,0.014233,-0.225601,-0.025627,0.075891
1,-0.006569,0.162164,-0.034963,0.000450,0.073238,-0.050382,-0.047629,-0.012692,-0.002964,1.643123,-0.091936,-0.006102,-0.182795,-0.123715,0.048619,-0.127930,0.022225,1.092707,-0.039193,-0.145075,-0.075526,-0.011026,-0.026827,0.042205,-0.019605,-0.129679,0.063317,0.070692,0.044736,-0.131742,-0.014476,0.118618,-0.070967,-0.079678,0.039858,-0.061898,-0.018870,-0.026671,0.045306,-0.066887,...,0.052841,0.008542,-0.040401,0.088016,0.081759,-0.059991,-0.079321,-0.034919,0.037969,-0.000143,0.116948,0.043643,-0.038576,-0.232962,-0.132695,0.038229,-0.018129,0.069727,0.037400,0.250792,0.190863,0.096718,-0.039025,-0.081061,-0.057699,0.014859,0.216638,0.063005,0.195241,-0.039588,-0.192229,0.104473,-0.016099,-0.161168,-0.093749,0.020272,0.059245,-0.153123,-0.042538,0.108835
2,0.067287,0.125487,-0.107641,-0.001247,0.097545,-0.097450,-0.045999,-0.008979,0.024508,1.576336,-0.064921,0.035447,-0.148896,-0.113620,-0.048472,-0.128786,-0.039658,1.033807,-0.026187,-0.189730,-0.076070,-0.087492,-0.073318,0.051909,-0.037713,-0.103671,0.053096,0.078343,0.063042,-0.098159,-0.025189,0.103880,0.014617,-0.030842,-0.001135,-0.009958,0.023751,0.012894,-0.022057,-0.107440,...,0.096346,0.053230,-0.037984,0.079541,0.087207,-0.059639,-0.052442,-0.094778,0.115241,0.007620,0.094813,0.051140,0.075257,-0.260254,-0.097349,-0.076774,0.054528,0.003379,0.038829,0.233990,0.182938,0.129455,-0.030463,-0.091245,-0.028677,-0.026665,0.241368,-0.024507,0.101604,-0.076269,-0.125711,0.030185,-0.004746,-0.132637,-0.067982,-0.003591,0.063082,-0.160280,-0.094235,0.112321
3,-0.063833,0.149846,-0.085239,-0.049047,0.024666,-0.034913,-0.142079,0.015872,0.035140,1.443637,-0.140130,0.053069,-0.235575,-0.114601,0.018573,-0.149369,0.065188,1.065974,-0.052207,-0.097630,-0.001791,-0.054468,0.053387,0.092118,-0.035336,-0.219713,0.035925,0.088149,0.002787,-0.205533,-0.104678,0.111119,0.063278,-0.145682,0.119764,-0.043513,-0.022543,-0.070984,-0.049478,0.051760,...,0.077383,0.002537,-0.004748,0.071880,-0.008416,-0.003428,-0.159312,-0.058375,-0.182992,0.053833,0.030334,0.000317,0.073139,-0.192469,-0.229101,0.023970,0.058906,0.124068,-0.040228,0.296762,0.131335,0.195511,0.016919,-0.120161,-0.021629,0.073011,0.300711,-0.002289,0.079986,-0.025342,-0.117617,0.062870,-0.016784,-0.118147,-0.142213,0.032802,0.112001,-0.275139,-0.032832,0.153854
4,-0.100422,0.158211,-0.207962,-0.061028,0.005256,0.028873,-0.191353,0.030922,0.043353,1.425559,-0.168916,0.051000,-0.275619,-0.110103,0.021150,-0.201462,-0.007365,1.066986,-0.064074,-0.069988,-0.029630,0.058498,-0.035926,0.061201,-0.025649,-0.209920,-0.005483,0.010188,0.030408,-0.378023,-0.068195,0.158896,0.040120,-0.173847,0.033720,-0.024462,0.003724,-0.029711,-0.155207,0.087034,...,0.089137,-0.038282,-0.015637,0.140685,-0.001261,0.021721,-0.112055,-0.078692,-0.072051,-0.067159,0.072476,0.052236,0.003156,-0.128124,-0.288746,0.108487,-0.002033,0.116187,0.045336,0.413025,0.242986,0.103016,0.114014,-0.200253,-0.009105,0.170552,0.364721,0

In [47]:
clf.fit(X_train_w2v, y_train)

AttributeError: ignored

In [29]:
parameters = {
    'max_depth':(5,10,15,20),
    'eta':(0.3, 0.5, 0.8)
}

grid_search = GridSearchCV(clf,parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train_w2v, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    2.8s finished


IndexError: ignored

In [ ]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

### Make a Submission File

In [65]:
# Predictions on test sample
pred = pipe.predict(X_test_w2v)

In [66]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [67]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [69]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'/content/drive/MyDrive/Colab Notebooks/Lambda DS/04.Unit_4/01.NLP/03.Document_Classification/submission{subNumber}.csv', index=False)
subNumber += 1

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?